# Sequence to Sequence를 이용한 Machine Translation

[https://tutorials.pytorch.kr/intermediate/seq2seq_translation_tutorial.html](https://tutorials.pytorch.kr/intermediate/seq2seq_translation_tutorial.html)

In [1]:
from __future__ import unicode_literals, print_function, division

import re
import time
import random
import unicodedata
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
plt.switch_backend('agg')

from io import open

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
DATA_PATH = "/home/pervinco/Datasets/en-fr/data/eng-fra.txt"
MAX_LENGTH = 100 ## 문장의 최대 길이.
TEACHER_FORCING_RATIO = 0.5

In [4]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {} ## 단어 : index
        self.word2count = {} ## 단어 등장횟수
        self.index2word = {0: "SOS", 1: "EOS"} ## index : 단어
        self.n_words = 2  ## 사전 내 단어 수. SOS 와 EOS 포함.

    def addSentence(self, sentence):
        """
        주어진 문장을 단어 단위로 분리하고, 각 단어를 사전에 추가.
        """
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [5]:
def unicodeToAscii(s):
    """
    유니 코드 문자열을 일반 ASCII로 변환한다.
        - unicodedata.normalize('NFD', s)는 문자열 s에서 결합된 문자(예: 악센트가 포함된 문자)를 기본 문자와 결합 마크로 분리한다.
        - 리스트 컴프리헨션([...])과 if unicodedata.category(c) != 'Mn' 조건을 사용하여, 결합 마크(Mn, Nonspacing Mark)를 제외한 문자만을 선택한다.
    """
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def normalizeString(s):
    """
    소문자화, 공백 제거, 아스키 변환, 그리고 비문자 제거
        - s.lower().strip()으로 문자열을 소문자로 변환하고 앞뒤 공백을 제거.
        - unicodeToAscii 함수를 호출하여 문자열을 아스키로 변환.
        - 정규 표현식 re.sub를 사용하여 문장부호는 유지하되, 알파벳과 기본 문장부호(.!?)를 제외한 모든 문자를 공백으로 대체한다.
    """

    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)

    return s

In [6]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    ## 파일을 읽고 개행 문자를("\n")기준으로 split
    lines = open(DATA_PATH, encoding='utf-8').read().strip().split('\n')
    print("===== Sample Lines =====")
    for line in lines[:5]:
        print(line)

    ## 모든 줄을 Tab("\t")을 기준으로 분리하고 normalizeString
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    print("===== Sample Pairs =====")
    for pair in pairs[:5]:
        print(pair)

    ## 쌍을 뒤집고, Lang 인스턴스 생성
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [7]:
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    """
    두 문장으로 구성된 쌍(p)을 입력으로 받아, 두 조건을 모두 만족하는지 확인.
        - len(p[0].split(' ')) < MAX_LENGTH와 len(p[1].split(' ')) < MAX_LENGTH : 두 문장의 길이가 MAX_LENGTH를 초과하지 않는지 검사.
        - p[1].startswith(eng_prefixes) : 쌍의 두 번째 문장(p[1])이 eng_prefixes 중 하나로 시작해야 한다.
    """
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH and p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    """
    문장 쌍의 리스트(pairs)를 입력으로 받아, filterPair 함수를 만족하는 쌍만을 필터링하여 새로운 리스트로 반환.
    각 쌍에 대해 filterPair 함수가 True를 반환하는 경우에만 해당 쌍을 결과 리스트에 포함시킨다.
    """
    return [pair for pair in pairs if filterPair(pair)]

In [8]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("\nRead %s sentence pairs" % len(pairs))

    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', True)
print(random.choice(pairs))

Reading lines...
===== Sample Lines =====
Go.	Va !
Run!	Cours !
Run!	Courez !
Wow!	Ça alors !
Fire!	Au feu !


===== Sample Pairs =====
['go .', 'va !']
['run !', 'cours !']
['run !', 'courez !']
['wow !', 'ca alors !']
['fire !', 'au feu !']

Read 135842 sentence pairs
Trimmed to 135842 sentence pairs
Counting words...
Counted words:
fra 21334
eng 13043
['j ai simplement fait ce que tu m as dit de faire .', 'i just did what you told me to do .']


In [9]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [10]:
class Decoder(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [11]:
def indexesFromSentence(lang, sentence):
    """
    주어진 문장을 공백을 기준으로 단어로 분리한 뒤, 각 단어에 해당하는 인덱스를 lang의 word2index 딕셔너리를 사용하여 찾아낸다. 
    결과적으로 문장을 인덱스의 배열로 변환.
    """
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    """
    문장을 인덱스 배열로 변환한 후, 특별한 토큰인 EOS_token (End Of Sentence 토큰, 문장의 끝을 나타냄)을 배열에 추가.
    배열을 Tensor로 변환.
    """
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [12]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False
    if use_teacher_forcing:
        # Teacher forcing 포함: 목표를 다음 입력으로 전달
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Teacher forcing 미포함: 자신의 예측을 다음 입력으로 사용
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # 입력으로 사용할 부분을 히스토리에서 분리

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [13]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # 주기적인 간격에 이 locator가 tick을 설정
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [14]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # print_every 마다 초기화
    plot_loss_total = 0  # plot_every 마다 초기화

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [15]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words


In [16]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [17]:
hidden_size = 256
encoder = Encoder(input_lang.n_words, hidden_size).to(device)
decoder = Decoder(hidden_size, output_lang.n_words).to(device)

trainIters(encoder, decoder, 100000, print_every=5000)

0m 31s (- 9m 49s) (5000 5%) 4.5000
1m 0s (- 9m 4s) (10000 10%) 4.0710
1m 29s (- 8m 27s) (15000 15%) 3.7995
1m 59s (- 7m 57s) (20000 20%) 3.5955
2m 29s (- 7m 27s) (25000 25%) 3.4156
2m 58s (- 6m 57s) (30000 30%) 3.2826
3m 28s (- 6m 26s) (35000 35%) 3.1637
3m 58s (- 5m 57s) (40000 40%) 3.1230
4m 27s (- 5m 27s) (45000 45%) 3.0148
4m 55s (- 4m 55s) (50000 50%) 2.9509
5m 25s (- 4m 26s) (55000 55%) 2.8967
5m 52s (- 3m 55s) (60000 60%) 2.8336
6m 22s (- 3m 26s) (65000 65%) 2.7996
6m 51s (- 2m 56s) (70000 70%) 2.7632
7m 21s (- 2m 27s) (75000 75%) 2.6983
7m 51s (- 1m 57s) (80000 80%) 2.6614
8m 21s (- 1m 28s) (85000 85%) 2.6238
8m 48s (- 0m 58s) (90000 90%) 2.6515
9m 14s (- 0m 29s) (95000 95%) 2.6037
9m 44s (- 0m 0s) (100000 100%) 2.5560


In [18]:
def evaluateAndShowAttention(input_sentence):
    output_words = evaluate(encoder, decoder, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))


evaluateAndShowAttention("elle a cinq ans de moins que moi .")

evaluateAndShowAttention("elle est trop petit .")

evaluateAndShowAttention("je ne crains pas de mourir .")

evaluateAndShowAttention("c est un jeune directeur plein de talent .")

input = elle a cinq ans de moins que moi .
output = she is five years than me . <EOS>
input = elle est trop petit .
output = she is too small . <EOS>
input = je ne crains pas de mourir .
output = i m not supposed of . <EOS>
input = c est un jeune directeur plein de talent .
output = it s a a of . . . <EOS>
